In [62]:
import spacy 
import numpy as np
import emb_spacy
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional, GRU
from keras.metrics import Accuracy, Precision, Recall, F1Score, MeanSquaredError




from emb_spacy import get_embedding
from input import count_examples_and_max_length, pad_sentences_from_file
import label

### Lecture des données 

In [2]:
# Ouverture en lecture des données
with open("train_corpus", "r", encoding="utf-8") as file:
    data = file.readlines()

# Compter les exemples et trouver la taille maximale
num_examples, MAX_SEQ_SIZE = count_examples_and_max_length(data)

# Affichage des résultats
print("Nombre d'exemples :", num_examples) 
print("Taille maximale de la phrase :", MAX_SEQ_SIZE)

vec_word, sortie = pad_sentences_from_file("train_corpus", MAX_SEQ_SIZE)
print("vecteur : ", vec_word.shape)
# vec_word.shape : (3945, 41)
# sortie.shape : (3945, 41)

Nombre d'exemples : 3945
Taille maximale de la phrase : 41
vecteur :  (3945, 41)


### Création des vecteurs d'entrée et de sortie

In [3]:
# Création du vecteur d'entrée 
entree = np.zeros((num_examples, MAX_SEQ_SIZE, 300))
for i, sentence in enumerate(vec_word):
    for j, word in enumerate(sentence):
        if word:
            entree[i, j] = get_embedding(word)
        else:
            entree[i, j] = np.zeros(300)

print("entree shape : ", entree.shape)
print("sortie shape : ", sortie.shape)

entree shape :  (3945, 41, 300)
sortie shape :  (3945, 41)


In [4]:
labels = label.extract_label("atis.train")
label_one_hot_dict = {label_: label.get_vector_from_label(label_) for label_ in labels} # création d'un dictionnaire associant chaque label à son vecteur one hot 

#print("LABELS :", labels)
nbLabels = len(labels)  # Nombre d'étiquettes potentielles
embedding_size = len(entree[0][0])

tailleDictionnaire = emb_spacy.get_size_dict()  # Taille du dictionnaire de mots

In [42]:
# print(sortie[0])
sortie_one_hot = np.zeros((sortie.shape[0], sortie.shape[1], len(label_one_hot_dict['O'])), dtype=int)
zero_vec = np.zeros(len(label_one_hot_dict['O']), dtype=int)
for i in range(sortie.shape[0]):
    for j in range(sortie.shape[1]):
        label = sortie[i, j]
        if label == '0':
            sortie_one_hot[i, j] = zero_vec
        else:
            sortie_one_hot[i, j] = label_one_hot_dict[label]

In [6]:
np.save('sortie_one_hot.npy', sortie_one_hot)

In [7]:
sortie[0]

array(['O', 'O', 'O', 'O', 'O', 'B-cost_relative', 'O', 'O',
       'B-fromloc.city_name', 'B-fromloc.state_code', 'O',
       'B-toloc.city_name', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0'], dtype=object)

In [8]:
sortie_one_hot.shape

(3945, 41, 81)

### Modèle LSTM

#### Configuration du modèle

In [66]:

config = {
    'hidden_size': 128, # Taille de la couche cachée du RNN
    'dropout_rate': 0.5,  # Taux de dropout
    'nb_labels': nbLabels
}  

# Définir l'entrée du modèle
# pas besoin de mettre le nb d'ex car il les fait passer un par un 
input_layer = Input(shape=(MAX_SEQ_SIZE, embedding_size), dtype='float32')

# Ajouter une couche LSTM bidirectionnelle
# X = Bidirectional(LSTM(units=config['hidden_size'], return_sequences=True))(input_layer)
X = LSTM(units=config['hidden_size'], return_sequences=True)(input_layer)
X = Dropout(config['dropout_rate'])(X)
# X = LSTM(units = 128)(X)
# X = Dropout(config['dropout_rate'])(X)

X = Dense(units=config['nb_labels'])(X)
X = Activation(activation='softmax')(X)
print('config ok')

# Création du model
model = Model(inputs=input_layer, outputs=X)
print("create ok")

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
# NB : j'ai enlevé sparse car j'ai fait un vecteur one hot en sortie et sparse c'est pour quand c'est pas des one hot
print("compil ok")
print(model.summary())

config ok
create ok
compil ok


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 41, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 41, 128)        │       219,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 41, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 41, 81)         │        10,449 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 41, 81)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 230,097 (898.82 KB)

 Trainable params: 230,097 (898.82 KB)

 Non-trainable params: 0 (0.00 B)

None


### Entraînement du modèle LSTM
- x : entree de la forme nb_example x MAX_SEQ_SIZE x embedding_size
- y : sortie de la forme nb_example x MAX_SEQ_SIZE
- batch_size : nb d'échantillon à utiliser à chaque itération lors de l'entrainement
    - un batch_size + gd = accélère l'entrainement mais besoin de + de mémoire GPU
    - plus petit = ralentit l'entrainement mais meilleure convergence du modèle
- epochs : nb d'itération sur l'ens des données d'entrainement 
- validation_split : spécifie le fraction des données à utiliser comme données de validation

In [28]:
model.fit(x=entree, y=sortie_one_hot, batch_size=128, epochs=10, validation_split=0.2)
print('training ok')

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 0.1538 - loss: 0.6997 - precision_2: 0.8035 - recall_2: 0.2023 - val_accuracy: 0.2166 - val_loss: 0.2126 - val_precision_2: 0.9383 - val_recall_2: 0.7457
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.3460 - loss: 0.2005 - precision_2: 0.9510 - recall_2: 0.7889 - val_accuracy: 0.9422 - val_loss: 0.1542 - val_precision_2: 0.9588 - val_recall_2: 0.8349
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.3895 - loss: 0.1378 - precision_2: 0.9750 - recall_2: 0.8378 - val_accuracy: 0.9725 - val_loss: 0.1302 - val_precision_2: 0.9602 - val_recall_2: 0.8541
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.3989 - loss: 0.1148 - precision_2: 0.9752 - recall_2: 0.8610 - val_accuracy: 0.9747 - val_loss: 0.1152 - val_precision_2: 0.9601 - val_recall_2: 0.8640
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.4299 - loss: 0.0979 - precision_2: 0.9781 - recall_2: 0.8796 - val

### Modèle GRU

#### Configuration du modèle

In [65]:

config = {
    'hidden_size': 128, # Taille de la couche cachée du RNN
    'dropout_rate': 0.5,  # Taux de dropout
    'nb_labels': nbLabels
}  

# Définir l'entrée du modèle
# pas besoin de mettre le nb d'ex car il les fait passer un par un 
input_layer = Input(shape=(MAX_SEQ_SIZE, embedding_size), dtype='float32')

# Ajouter une couche LSTM bidirectionnelle
# X = Bidirectional(LSTM(units=config['hidden_size'], return_sequences=True))(input_layer)
X = GRU(units=config['hidden_size'], return_sequences=True)(input_layer)
X = Dropout(config['dropout_rate'])(X)
# X = LSTM(units = 128)(X)
# X = Dropout(config['dropout_rate'])(X)

X = Dense(units=config['nb_labels'])(X)
X = Activation(activation='softmax')(X)
print('config ok')

# Création du model
model_GRU = Model(inputs=input_layer, outputs=X)
print("create ok")

# Compilation du modèle
model_GRU.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
# NB : j'ai enlevé sparse car j'ai fait un vecteur one hot en sortie et sparse c'est pour quand c'est pas des one hot
print("compil ok")
print(model_GRU.summary())

config ok
create ok
compil ok


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 41, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 41, 128)        │       165,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 41, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 41, 81)         │        10,449 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 41, 81)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 175,569 (685.82 KB)

 Trainable params: 175,569 (685.82 KB)

 Non-trainable params: 0 (0.00 B)

None


#### Entrainement du modèle

In [67]:
model_GRU.fit(x=entree, y=sortie_one_hot, batch_size=128, epochs=10, validation_split=0.2)
print('training ok')

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.1819 - loss: 0.7274 - precision_5: 0.8308 - recall_5: 0.2560 - val_accuracy: 0.9328 - val_loss: 0.2337 - val_precision_5: 0.9486 - val_recall_5: 0.7355
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.4066 - loss: 0.2476 - precision_5: 0.9321 - recall_5: 0.7261 - val_accuracy: 0.9667 - val_loss: 0.1817 - val_precision_5: 0.9607 - val_recall_5: 0.7930
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.4312 - loss: 0.1937 - precision_5: 0.9434 - recall_5: 0.7817 - val_accuracy: 0.9704 - val_loss: 0.1605 - val_precision_5: 0.9677 - val_recall_5: 0.8180
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.4220 - loss: 0.1669 - precision_5: 0.9521 - recall_5: 0.8090 - val_accuracy: 0.9738 - val_loss: 0.1434 - val_precision_5: 0.9710 - val_recall_5: 0.8396
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.4234 - loss: 0.1499 - precision_5: 0.9588 - recall_5: 0.8284 - val

### Prétraitement des données de test

In [29]:
with open("test_corpus", "r", encoding="utf-8") as file:
    data_test = file.readlines()
    
num_examples_test, MAX_SEQ_SIZE_TEST = count_examples_and_max_length(data_test)


vec_word_test, label_test_real = pad_sentences_from_file("test_corpus", MAX_SEQ_SIZE)

print("Nombre d'exemples :", num_examples_test) 
print("Taille maximale de la phrase :", MAX_SEQ_SIZE_TEST)

# Création du vecteur d'entrée 
test_data_input = np.zeros((num_examples_test, MAX_SEQ_SIZE, 300))
for i, sentence in enumerate(vec_word_test):
    for j, word in enumerate(sentence):
        if word:
            test_data_input[i, j] = get_embedding(word)
        else:
            test_data_input[i, j] = np.zeros(300)

Nombre d'exemples : 1033
Taille maximale de la phrase : 32


In [30]:
print(test_data_input.shape)
print(label_test_real.shape)
print(entree.shape)

(1033, 41, 300)
(1033, 41)
(3945, 41, 300)


In [45]:
label_test_real_one_hot = np.zeros((label_test_real.shape[0], label_test_real.shape[1], len(label_one_hot_dict['O'])), dtype=int)
zero_vec = np.zeros(len(label_one_hot_dict['O']), dtype=int)
for i in range(label_test_real.shape[0]):
    for j in range(label_test_real.shape[1]):
        label = label_test_real[i, j]
        if label == '0':
            label_test_real_one_hot[i, j] = zero_vec
        else:
            label_test_real_one_hot[i, j] = label_one_hot_dict[label]

print(label_test_real_one_hot.shape)

(1033, 41, 81)


### Prédictions

#### Avec le modèle LSTM

In [31]:
test_predictions = model.predict(test_data_input)

print(test_predictions.shape)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
(1033, 41, 81)


In [46]:
predicted_labels_index = np.argmax(test_predictions, axis=-1)
print(predicted_labels_index[10])
predicted_labels = [[labels[idx] for idx in sample] for sample in predicted_labels_index]
predicted_labels = np.array(predicted_labels)
print(predicted_labels[10])

[0 0 0 0 0 2 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
['O' 'O' 'O' 'O' 'O' 'B-fromloc.city_name' 'O' 'B-toloc.city_name' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']


In [54]:
np.save('predicted_labels.txt', predicted_labels)

In [52]:
print(label_test_real[10])

['O' 'O' 'O' 'O' 'O' 'B-fromloc.city_name' 'O' 'B-toloc.city_name' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']


In [58]:
_ = model.evaluate(test_data_input, label_test_real_one_hot)
# print("Loss on test data:", test_loss)
# print("Accuracy on test data:", test_accuracy)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9816 - loss: 0.0842 - precision_2: 0.9691 - recall_2: 0.8999
[[[  1.86109996   6.47429991  -5.76140022 ...  -0.65935999  -4.36250019
     2.023     ]
  [  2.13870001  -2.89310002  -2.99799991 ...  -3.12260008 -12.38399982
     3.29469991]
  [ -2.2658999    4.19360018  -6.72590017 ...  -5.37200022  -2.89650011
    -0.047368  ]
  ...
  [  2.16750002  -5.44159985   7.14239979 ...  -3.54999995   6.75159979
     1.14100003]
  [  2.16750002  -5.44159985   7.14239979 ...  -3.54999995   6.75159979
     1.14100003]
  [  2.16750002  -5.44159985   7.14239979 ...  -3.54999995   6.75159979
     1.14100003]]

 [[ -2.48320007   4.02440023  -5.4454999  ...   0.92074001  -1.54569995
     3.25990009]
  [ -7.13950014  -2.43390012  -2.51810002 ...  -4.81470013  -5.17789984
     6.76970005]
  [ -5.10430002   2.34960008   3.24720001 ...  -7.6875      -2.51279998
     0.69341999]
  ...
  [  2.16750002  -5.44159985   7.14239979 ...  -3.54999995   6.7515997

#### Avec le modèle GRU

In [68]:
test_predictions_GRU = model_GRU.predict(test_data_input)

print(test_predictions_GRU.shape)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
(1033, 41, 81)


In [69]:
_ = model_GRU.evaluate(test_data_input, label_test_real_one_hot)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9769 - loss: 0.1220 - precision_5: 0.9643 - recall_5: 0.8771
